# Describe Individual batch's network

Definition of individual batch's network, looping through years: 
- Year i to i+1's campaign financial data
- Year i+2 to i+3's bill co-authorship data

For every term (finance + bill - one mode projection on to candidate):
- Avg clustering coefficient
- WCC size
- SCC size
- Max path length
- Average node degree


In [4]:
import snap
import numpy as np
import pandas as pd
from itertools import permutations
from matplotlib import pyplot as plt
import random
import glob

CAM_YEAR = 1982
TERM_YEAR = 1984
TERM_START = 98 #1983-1984


##Secion 1 - Load Candidate Attributes

In [16]:
filePath = "../processed-data/candidate_node_mapping_manual.csv"
mapping = pd.read_csv(filePath, index_col=False)

filePath = "../data/financials/candidate_master/"
candFile = filePath + str(CAM_YEAR) + ".txt"

cand = pd.read_csv(candFile, sep='|', index_col=False, 
                 names=['CAND_ID',
                 'CAND_NAME',
                 'CAND_PTY_AFFILIATION',
                 'CAND_ELECTION_YR',
                 'CAND_OFFICE_ST',
                 'state',
                 'CAND_OFFICE',
                 'CAND_OFFICE_DISTRICT',
                 'CAND_ICI',
                 'CAND_STATUS',
                 'CAND_PCC',
                 'CAND_ST1',
                 'CAND_ST2',
                 'CAND_CITY',
                 'CAND_ST'])
cand = cand[['CAND_ID','CAND_NAME', 'CAND_PTY_AFFILIATION', 
             'CAND_ELECTION_YR','CAND_OFFICE_ST','state','CAND_OFFICE','CAND_ST2']]

cand['CAM_YEAR'] = CAM_YEAR

c = pd.merge(cand, mapping, on='CAND_ID', how='left')

c[['NodeID']] = c[['NodeID']].apply(pd.to_numeric, errors='coerce').fillna(-1).astype(np.int64)
   
## remove entries where Nid = 0, meaning unsuccessful candidates
c = c[c.fname != '']
c = c[c.NodeID != -1]


print c.head()

     CAND_ID             CAND_NAME_x CAND_PTY_AFFILIATION  CAND_ELECTION_YR  \
2  H0AS00018  HUNKIN, ENI FA'AUAA JR                  IND              1980   
3  H0AS00018  HUNKIN, ENI FA'AUAA JR                  IND              1980   
4  H0AS00018  HUNKIN, ENI FA'AUAA JR                  IND              1980   
5  H0AS00018  HUNKIN, ENI FA'AUAA JR                  IND              1980   
6  H0AS00026         SUNIA, FOFO I F                  DEM              1982   

  CAND_OFFICE_ST state_x  CAND_OFFICE    CAND_ST2  CAM_YEAR  \
2             AS       H            0   PAGO PAGO      1982   
3             AS       H            0   PAGO PAGO      1982   
4             AS       H            0   PAGO PAGO      1982   
5             AS       H            0   PAGO PAGO      1982   
6             AS       H            0  WASHINGTON      1982   

                         CAND_NAME_y state_y         lname fname  \
2                  FALEOMAVAEGA, ENI      AS  FALEOMAVAEGA   ENI   
3         